In [1]:
import json
import numpy as np
import pandas as pd
import os

In [2]:
def convert_str_to_float(string):
    num, denom = string.split('/')
    return float(num)/float(denom)

In [3]:
DATA_PATH = "data/1/"

In [4]:

content_list = []

for content in os.listdir(DATA_PATH + "votes"):
    if content != '.DS_Store':
        content_list.append(content)

for year in content_list:
    house_count = 0
    senate_count = 0
    path = DATA_PATH + "votes/" + year
    for folder in os.listdir(path):
        if folder[0] == 'h':
            house_count += 1
        elif folder[0] == 's':
            senate_count += 1
print house_count
print senate_count

51
61


In [5]:
res = []
#I might be interested in votes with categories like ['passage-suspension', 'treaty', 'conviction', 'veto-override']
for year in content_list:
    for i in range(1, senate_count+1):
        path = DATA_PATH + 'votes/' + year + '/s' + str(i) + '/data.json'
        f = open(path, 'r')
        x = json.loads(f.read())
        if x['category'] == 'passage' or x['category'] == 'amendment':
            res.append(i)

IOError: [Errno 2] No such file or directory: 'data/1/votes/3/s1/data.json'

In [26]:
print res
init_dict = True
votes = ['Yea', 'Nay', 'Present', 'Not Voting']
for index in res:
    path = DATA_PATH + 'votes/2014/s' + str(index) + '/data.json'
    vote = json.loads(open(path, 'r').read())
    if init_dict:
        vote_dict = {}
        vote_dict['date'] = [vote['date']]
#         vote_dict['billTitle'] = [vote['subject']]
        if vote['category'] == 'passage':
            vote_dict['title'] = [vote['bill']['type'] + str(vote['bill']['number'])]
            vote_dict['isAmendment'] = [0]
        else:
            vote_dict['title'] = [vote['amendment']['type'] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number'])]
            vote_dict['isAmendment'] = [1]
        vote_dict['result'] = [vote['result_text']]
        vote_dict['requires'] = [convert_str_to_float(vote['requires'])]
        for vote_type in votes:
            for person in vote['votes'][vote_type]:
                vote_dict[person['display_name']] = [vote_type]
        init_dict = False
    else:
        vote_dict['date'].append(vote['date'])
#         vote_dict['billTitle'].append(vote['subject'])
        if vote['category'] == 'passage':
            vote_dict['title'].append(vote['bill']['type'] + str(vote['bill']['number']))
            vote_dict['isAmendment'].append(0)
        else:
            vote_dict['title'].append(vote['amendment']['type'] + "amdt" + str(vote['amendment']['number']) + " to " + vote['bill']['type'] + str(vote['bill']['number']))
            vote_dict['isAmendment'].append(1)
        vote_dict['result'].append(vote['result_text'])
        vote_dict['requires'].append(convert_str_to_float(vote['requires']))
        for vote_type in votes:
            for person in vote['votes'][vote_type]:
                if person['display_name'] in vote_dict:
                     vote_dict[person['display_name']].append(vote_type)
                else:
                    vote_dict[person['display_name']] = [np.nan]*(len(vote_dict['isAmendment'])-1) + [vote_type]

#pad with nones to make all lists the same size

for key in vote_dict:
    if len(vote_dict[key]) != len(vote_dict['isAmendment']):
        vote_dict[key] = vote_dict[key] + [np.nan]*(len(vote_dict['isAmendment']) - len(vote_dict[key]))

import pandas as pd
df = pd.DataFrame(data=vote_dict)
#df.to_csv('rollCallVotes_iter1.csv', index=False)

[11, 13, 16, 17, 18, 19, 21, 34, 35, 62, 72, 73, 74, 75, 77, 78, 88, 93, 101, 163, 187, 212, 213, 214, 230, 231, 244, 245, 246, 247, 248, 254]
